In [1]:
import pandas as pd
import numpy as np
import spacy
import re
import requests
from bs4 import BeautifulSoup

C:\Users\spenc\anaconda3\envs\textprocessing2\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
path = 'https://www.cisa.gov/uscert/ncas/alerts.xml'

In [6]:
d = feedparser.parse(path)
d['feed']['title']

'CISA Alerts'

In [28]:
d.feed.title

'CISA Alerts'

In [36]:
d.feed.link

'https://us-cert.cisa.gov/'

In [34]:
d.feed.description

'Alerts warn about vulnerabilities, incidents, and other security issues that pose a significant risk.'

In [39]:
d.entries[0].title

'AA22-294A: #StopRansomware: Daixin Team'

In [23]:
d.entries[0].link

'https://us-cert.cisa.gov/ncas/alerts/aa22-294a'

In [24]:
d.entries[0].description[:100]

'Original release date: October 21, 2022 | Last revised: October 26, 2022<br /><h3>Summary</h3><p cla'

In [25]:
d.entries[0].published

'Fri, 21 Oct 2022 14:29:15 +0000'

In [26]:
d.entries[0].published_parsed

time.struct_time(tm_year=2022, tm_mon=10, tm_mday=21, tm_hour=14, tm_min=29, tm_sec=15, tm_wday=4, tm_yday=294, tm_isdst=0)

In [43]:
d.entries[0].id

'18087 at https://us-cert.cisa.gov'

# get alerts beautifulsoup

In [2]:
import requests
from bs4 import BeautifulSoup

In [3]:
url='https://www.cisa.gov/uscert/ncas/alerts'
page = requests.get(url)
soup = BeautifulSoup(page.text, 'html.parser')

In [23]:
soup

<!DOCTYPE html>

<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
<head>
<meta charset="utf-8"/>
<script async="" src="/sites/default/files/google_analytics/gtag.js?rljaka"></script>
<script>window.dataLayer = window.dataLayer || [];function gtag(){dataLayer.push(arguments)};gtag("js", new Date());gtag("set", "developer_id.dMDhkMT", true);gtag("config", "UA-34879253-1", {"groups":"default","anonymize_ip":true,"page_placeholder":"PLACEHOLDER_page_path","allow_ad_personalization_signals":false});</script>
<meta content="As part of its mission, CISA leads the effort to enhance the security, resiliency, and reliability of the Nation's cybersecur

In [198]:
alert_url='https://www.cisa.gov/uscert/ncas/alerts/aa22-321a'
alert_page = requests.get(alert_url)
alert_soup = BeautifulSoup(alert_page.text, 'html.parser')
soup

<!DOCTYPE html>

<html dir="ltr" lang="en" prefix="content: http://purl.org/rss/1.0/modules/content/  dc: http://purl.org/dc/terms/  foaf: http://xmlns.com/foaf/0.1/  og: http://ogp.me/ns#  rdfs: http://www.w3.org/2000/01/rdf-schema#  schema: http://schema.org/  sioc: http://rdfs.org/sioc/ns#  sioct: http://rdfs.org/sioc/types#  skos: http://www.w3.org/2004/02/skos/core#  xsd: http://www.w3.org/2001/XMLSchema# ">
<head>
<meta charset="utf-8"/>
<script async="" src="/sites/default/files/google_analytics/gtag.js?rljaka"></script>
<script>window.dataLayer = window.dataLayer || [];function gtag(){dataLayer.push(arguments)};gtag("js", new Date());gtag("set", "developer_id.dMDhkMT", true);gtag("config", "UA-34879253-1", {"groups":"default","anonymize_ip":true,"page_placeholder":"PLACEHOLDER_page_path","allow_ad_personalization_signals":false});</script>
<meta content="As part of its mission, CISA leads the effort to enhance the security, resiliency, and reliability of the Nation's cybersecur

In [192]:
#get content for each alert on the first 10 pages of alerts
alerts = []
for p_num in range(10): #look at 10 pages of results
    #get the the content from this page
    url='https://www.cisa.gov/uscert/ncas/alerts?page=' + str(p_num)
    page = requests.get(url)
    soup = BeautifulSoup(page.text, 'html.parser')
    table = soup.find_all(class_="views-field views-field-title")
    
    for i in range(len(table)):
        #get the alert details from this line in the list
        link = table[i].find('a')['href']
        alert_id = table[i].find(class_="field-content").contents[0]
        title = table[i].find('a').text
        
        #get alert content
        alert_url='https://www.cisa.gov/uscert' + link
        alert_page = requests.get(alert_url)
        alert_soup = BeautifulSoup(alert_page.text, 'html.parser')
        alert_date = alert_soup.find(id="ncas-header").find(class_="submitted meta-text").text
        alert_text = alert_soup.find(id="ncas-content").text
        
        #write the content
        alerts.append([link, alert_id, title, alert_date, alert_text])

In [239]:
headers = ['link', 'alert_id', 'title', 'date', 'text']
df = pd.DataFrame(alerts, columns=headers)
df.head()

,link,alert_id,title,date,text
0,/ncas/alerts/aa22-321a,AA22-321A :,#StopRansomware: Hive Ransomware,\n Original rel...,\n\nSummary\nActions to Take Today to Mitigate...
1,/ncas/alerts/aa22-320a,AA22-320A :,Iranian Government-Sponsored APT Actors Compro...,\n Original rel...,\n\nSummary\nFrom mid-June through mid-July 20...
2,/ncas/alerts/aa22-294a,AA22-294A :,#StopRansomware: Daixin Team,\n Original rel...,\n\nSummary\nActions to take today to mitigate...
3,/ncas/alerts/aa22-279a,AA22-279A :,Top CVEs Actively Exploited By People’s Republ...,\n ...,\n\nSummary\nThis joint Cybersecurity Advisory...
4,/ncas/alerts/aa22-277a,AA22-277A :,Impacket and Exfiltration Tool Used to Steal S...,\n Original rel...,\n\nSummary\nActions to Help Protect Against A...


# parse alert content

In [1]:
re_cve = re.compile(r'(CVE-\d{4}-\d*)')
re_cve.findall(alerts[0][4])

NameError: name 're' is not defined

In [229]:
#regex for cve match
re_cve = re.compile(r'(CVE-\d{4}-\d*)')
cves = []
for i in range(len(alerts)):
    alert_cves = re_cve.findall(alerts[i][4])
    cves.append(alert_cves)

In [240]:
#add list of cve to df
df['CVE_list'] = cves
df.head()

,link,alert_id,title,date,text,CVE_list
0,/ncas/alerts/aa22-321a,AA22-321A :,#StopRansomware: Hive Ransomware,\n Original rel...,\n\nSummary\nActions to Take Today to Mitigate...,"[CVE-2020-12812, CVE-2021-31207, CVE-2021-3447..."
1,/ncas/alerts/aa22-320a,AA22-320A :,Iranian Government-Sponsored APT Actors Compro...,\n Original rel...,\n\nSummary\nFrom mid-June through mid-July 20...,[CVE-2021-44228]
2,/ncas/alerts/aa22-294a,AA22-294A :,#StopRansomware: Daixin Team,\n Original rel...,\n\nSummary\nActions to take today to mitigate...,[]
3,/ncas/alerts/aa22-279a,AA22-279A :,Top CVEs Actively Exploited By People’s Republ...,\n ...,\n\nSummary\nThis joint Cybersecurity Advisory...,"[CVE-2021-44228, CVE-2019-11510, CVE-2021-2220..."
4,/ncas/alerts/aa22-277a,AA22-277A :,Impacket and Exfiltration Tool Used to Steal S...,\n Original rel...,\n\nSummary\nActions to Help Protect Against A...,"[CVE-2021-26855, CVE-2021-26857, CVE-2021-2685..."


# NER

In [ ]:
nlp = spacy.load("en_core_web_trf")

In [264]:
#extract entity from each text
ner = []
for i in range(len(alerts)):
    #nlp this doc
    this_doc = nlp(alerts[i][4])
    
    #select get entities from this doc
    this_ner = []
    for ent in this_doc.ents:
        #choose from list of 
        ent_labels = ['ORG','PRODUCT','GPE','PERSON']
        if ent.label_ in ent_labels:
            this_ner.append([ent.text, ent.label_])
        
    #append ner for this doc
    ner.append(this_ner)

Token indices sequence length is longer than the specified maximum sequence length for this model (779 > 512). Running this sequence through the model will result in indexing errors


In [265]:
#add ner into df. leave duplicates for other opportunity
df['ner']=ner

In [266]:
df.to_pickle("./alerts.pkl")  

# clean for graph

In [2]:
df = pd.read_pickle("./alerts.pkl") 

In [3]:
df.head()

,link,alert_id,title,date,text,CVE_list,ner
0,/ncas/alerts/aa22-321a,AA22-321A :,#StopRansomware: Hive Ransomware,\n Original rel...,\n\nSummary\nActions to Take Today to Mitigate...,"[CVE-2020-12812, CVE-2021-31207, CVE-2021-3447...","[[stopransomware.gov, ORG], [The Federal Burea..."
1,/ncas/alerts/aa22-320a,AA22-320A :,Iranian Government-Sponsored APT Actors Compro...,\n Original rel...,\n\nSummary\nFrom mid-June through mid-July 20...,[CVE-2021-44228],"[[CISA, ORG], [Federal Civilian Executive Bran..."
2,/ncas/alerts/aa22-294a,AA22-294A :,#StopRansomware: Daixin Team,\n Original rel...,\n\nSummary\nActions to take today to mitigate...,[],"[[The Federal Bureau of Investigation, ORG], [..."
3,/ncas/alerts/aa22-279a,AA22-279A :,Top CVEs Actively Exploited By People’s Republ...,\n ...,\n\nSummary\nThis joint Cybersecurity Advisory...,"[CVE-2021-44228, CVE-2019-11510, CVE-2021-2220...","[[People’s Republic of China, GPE], [PRC, GPE]..."
4,/ncas/alerts/aa22-277a,AA22-277A :,Impacket and Exfiltration Tool Used to Steal S...,\n Original rel...,\n\nSummary\nActions to Help Protect Against A...,"[CVE-2021-26855, CVE-2021-26857, CVE-2021-2685...",[[the Cybersecurity and Infrastructure Securit...


In [4]:
#clean alert id
df['alert_id'] = df['alert_id'].str.replace(' :','').str.strip()

#extract date from date field


#convert link to full path
df['full_path'] = 'https://www.cisa.gov' + df['link']


In [5]:
print(df.iloc[[0],[3]])

                                                date
0  \n                                Original rel...


# alert to CVE edges

In [5]:
#alert to cve edges
alert_cve = df[['alert_id','CVE_list']].explode('CVE_list')
alert_cve = alert_cve.drop_duplicates().dropna()
alert_cve.to_csv('./alert_cve_edge.csv')

# alert nodes

In [11]:
df[['alert_id','full_path','title','date']].drop_duplicates().to_csv('./alert_nodes.csv')

# dedupe

In [6]:
import os
import csv
import re
import logging
import optparse

import dedupe
from unidecode import unidecode

In [7]:
#alert to entity edges
df_ner = df[['alert_id','ner']].explode('ner')
df_ner[['label','type']] = pd.DataFrame(df_ner.ner.tolist(), index= df_ner.index)
df_ner=df_ner.reset_index(level=0)
df_ner=df_ner.drop(columns='ner')

In [8]:
ner_dedupe = df_ner[['label', 'type']]
ner_dedupe=ner_dedupe.drop_duplicates()
ner_dedupe=ner_dedupe.reset_index()
ner_dedupe.to_csv('./alert_ner.csv', index=True, index_label = 'Id', columns=['label','type'])

In [9]:
#Do a little bit of data cleaning with the help of Unidecode and Regex. Things like casing, extra spaces, quotes and new lines can be ignored.

def preProcess(column):
#
    column = unidecode(column)
    column = re.sub('  +', ' ', column)
    column = re.sub('\n', ' ', column)
    column = column.strip().strip('"').strip("'").lower().strip()
#If data is missing, indicate that by setting the value to None

    if not column:
        column = None
    return column
#Read in our data from a CSV file and create a dictionary of records, where the key is a unique record ID and each value is dict

def readData(filename):
#
    data_d = {}
    with open(filename, encoding="utf8") as f:
        reader = csv.DictReader(f)
        for row in reader:
            clean_row = [(k, preProcess(v)) for (k, v) in row.items()]
            row_id = int(row['Id'])
            data_d[row_id] = dict(clean_row)

    return data_d


In [10]:
input_file = 'alert_ner.csv'
output_file = 'ner_cluster.csv'
training_file = 'ner_training.json'

In [11]:
data_d=readData(input_file)

In [12]:
#define dedupe
fields = [
            {'field': 'label', 'type': 'String'},            {'field': 'type', 'type': 'String'}
            ]
deduper = dedupe.Dedupe(fields)

In [13]:
print('preparing training...')
if os.path.exists(training_file):
    print('reading labeled examples from ', training_file)
    with open(training_file, 'rb') as f:
        deduper.prepare_training(data_d, f)
else:
    deduper.prepare_training(data_d)

preparing training...


In [27]:
print('starting active labeling...')
dedupe.console_label(deduper)

label : windows nt
type : product

label : windows backup manager
type : product

52/10 positive, 52/10 negative
Do these records refer to the same thing?
(y)es / (n)o / (u)nsure / (f)inished


starting active labeling...


 f


Finished labeling


In [28]:
deduper.train()

with open(training_file, 'w') as tf:
        deduper.write_training(tf)

NoIndexError: Attempting to block with an index predicate without indexing records

In [22]:
print('clustering...')
clustered_dupes = deduper.partition(data_d, 0.5)

print('# duplicate sets', len(clustered_dupes))

clustering...
# duplicate sets 945


In [23]:
cluster_membership = {}
for cluster_id, (records, scores) in enumerate(clustered_dupes):
    for record_id, score in zip(records, scores):
        cluster_membership[record_id] = {
            "Cluster ID": cluster_id,
            "confidence_score": score
        }

with open(output_file, 'w') as f_output, open(input_file, encoding="utf8") as f_input:

    reader = csv.DictReader(f_input)
    fieldnames = ['Cluster ID', 'confidence_score'] + reader.fieldnames

    writer = csv.DictWriter(f_output, fieldnames=fieldnames)
    writer.writeheader()

    for row in reader:
        row_id = int(row['Id'])
        row.update(cluster_membership[row_id])
        writer.writerow(row)

# alert to ner edge

In [69]:
df_clusters = pd.read_csv('./ner_cluster_id.csv')

In [38]:
df_ner=df_ner.merge(df_clusters, left_on='index', right_on='Id')

In [59]:
alert_ner_edge = pd.DataFrame(df_ner.groupby(by=['alert_id','label']).size())

In [67]:
alert_ner_edge = alert_ner_edge.reset_index()
alert_ner_edge.to_csv('./alert_ner_edge.csv', index=False, header=['alert_id','ner','weight'])

# ner node

In [76]:
ner_node = df_ner.groupby(by=['label','Cluster ID']).size()
ner_node = ner_node.reset_index()
ner_node.to_csv('./ner_node.csv', index=False, header=['ner','cluster id','weight'])